# DRACO - Milestone 2: Dataset exploration

This document is structured as follow:

1. Characters Data - Extraction and Processing
2. Movie Data - Extraction and Processing
3. Actors Ethinicites - Exploration

---

In [317]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [318]:
DATA_FOLDER = './Data/'

CHARACTER_PATH = DATA_FOLDER + 'MovieSummaries/character.metadata.tsv'
MOVIE_PATH = DATA_FOLDER + 'MovieSummaries/movie.metadata.tsv'
ETHNICITY_PATH = DATA_FOLDER + 'ethnicities_data.tsv'
ETHNICITY_GROUP_PATH = DATA_FOLDER + 'ethnicity_group_data.tsv'
NAME_PATH = DATA_FOLDER + 'MovieSummaries/name.clusters.txt'
PLOT_PATH = DATA_FOLDER + 'MovieSummaries/plot_summaries.txt'

## Importation of each datasets

### Character data

In [319]:
characters_original = pd.read_csv(CHARACTER_PATH, sep='\t', header=None, 
    names = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie release date", "Character name", "Birth", 
    "Gender", "Height", "Ethnicity ID", "Name", "Age at movie release",
    "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"])

### Movie data

In [320]:
movies_original = pd.read_csv(MOVIE_PATH, sep='\t', header=None, 
    names = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie name","Movie release date", "Box office revenue","Movie runtime","Movie language","Movie countries","Movie genres" ])

### Ethnicity data

In [321]:
ethnicities_original = pd.read_csv(ETHNICITY_PATH, sep='\t',  
                               header=0, names=["Ethnicity ID", "Ethnicity"])

ethnic_group_original = pd.read_csv(ETHNICITY_GROUP_PATH, sep='\t')

## Preprocessing of the Data

### Cleaning the data

#### Character data

Let's see what the movie dataset looks like.

In [322]:
characters_original.head()

,Wikipedia Movie ID,Freebase Movie ID,Movie release date,Character name,Birth,Gender,Height,Ethnicity ID,Name,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [323]:
characters_original.describe()

,Wikipedia Movie ID,Height,Age at movie release
count,4.506690e+05,154824.000000,292556.000000
mean,1.396975e+07,1.788893,37.788523
std,1.079662e+07,4.379940,20.587870
min,3.300000e+02,0.610000,-7896.000000
25%,3.759292e+06,1.676400,28.000000
50%,1.189065e+07,1.750000,36.000000
75%,2.366501e+07,1.830000,47.000000
max,3.750192e+07,510.000000,103.000000


As wee can see there is issues in the dataset. 
- First some ethinicites as well as gender are not specified.
- Second we can see that the age at movie release spans from -7896 to 103 years.

For our analysis we have decided to discard all characters that don't have a specified ethnicity as well as a specified gender. Moreover, only strictly positive ages will be taken into account. Also, since we are not interrested in the height and name of the actor that the column `Freebase Movie ID` is redundant with `Wikipedia Movie ID`, we have decided to drop those columns. 

In [324]:
characters = characters_original.copy()
characters = characters.drop(['Freebase Movie ID','Height','Name'], axis=1)

In [325]:
characters = characters[characters['Ethnicity ID'].notna()]
characters = characters[characters['Gender'].notna()]
characters = characters[characters['Age at movie release'] > 0]

In [326]:
characters.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
1,975900,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,/m/044038p,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
5,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft
11,975900,2001-08-24,Tres,1959-03-09,M,/m/064b9n,42.0,/m/0bgchrs,/m/0bgchrw,/m/03ydsb
27,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp


#### Movie data

Let's see what the movie dataset looks like.

In [327]:
movies_original.head()

,Wikipedia Movie ID,Freebase Movie ID,Movie name,Movie release date,Box office revenue,Movie runtime,Movie language,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [328]:
movies_original.describe()

,Wikipedia Movie ID,Box office revenue,Movie runtime
count,8.174100e+04,8.401000e+03,6.129100e+04
mean,1.740784e+07,4.799363e+07,1.118192e+02
std,1.098791e+07,1.121753e+08,4.360070e+03
min,3.300000e+02,1.000000e+04,0.000000e+00
25%,7.323695e+06,2.083193e+06,8.100000e+01
50%,1.777899e+07,1.063969e+07,9.300000e+01
75%,2.715573e+07,4.071696e+07,1.060000e+02
max,3.750192e+07,2.782275e+09,1.079281e+06


As wee can see there is issues in the dataset. 
- First some movies' box office revenue are not specified. 
- Second we can see that the movie run time spans from zero minutes to more than one million of minutes.

For our analysis we have decided to discard all movies that don't have a specified box office revenue and all movie that are shorter than ten minutes. Also, since we are not interrested in the movie language and that the column `Freebase Movie ID` is redundant with `Wikipedia Movie ID`, we have decided to drop this column. 

In [329]:
movies = movies_original.copy()
movies = movies.drop(['Freebase Movie ID','Movie language'], axis=1)

We can observe that the columns `Movie countries` and `Movie genres` contain dictionaries. In our case, it would be much more convenient to have lists instead. Let's process it accordingly.

In [330]:
movies["Movie countries"] = movies["Movie countries"].apply(lambda x: list(json.loads(x).values()) if len(json.loads(x).values()) > 0 else 'NaN')
movies["Movie genres"] = movies["Movie genres"].apply(lambda x: list(json.loads(x).values()) if len(json.loads(x).values()) > 0 else 'NaN')

As the dataset was realesed on 2013, the data from this year are not complet and thus should be removed. To simplify this step we can add a category for the year of release in addition of the date of release.

In [331]:
#Errors = 'coerce' will force the values that are outside the bound to be NaT
movies["Movie release year"] = pd.to_datetime(movies["Movie release date"],format='mixed',errors = 'coerce').dt.year
#Remove the NaN and NaT values
movies = movies[movies["Movie release year"].notna()]
#Express all years of realese as int.
movies["Movie release year"] = movies["Movie release year"].astype("int")

In [332]:
movies = movies[movies['Movie release year'] < 2013]
movies = movies[movies['Box office revenue'].notna()]
movies = movies[movies['Movie runtime'] >= 10]

We should now pay attention to remove the movie of the Animation field as the actors that have played inside are rather voice actors than actors. Generally in this domain, the looking and the ethinicty of the actors isn't as relevant as in the real action movies.

In [333]:
# First let's see all the unique genres, to discard the animation ones:
unique_genres = set()
movies['Movie genres'].apply(lambda x: unique_genres.update(x))
print(unique_genres)

{'Culture & Society', 'Screwball comedy', 'Monster', 'Comedy Thriller', 'Religious Film', 'Biopic [feature]', 'Erotic Drama', 'Action', 'Epic', 'Comedy', 'School story', 'Psychological horror', 'Albino bias', 'Disaster', 'Jukebox musical', 'Alien invasion', 'Historical Epic', 'Buddy film', 'Docudrama', 'Spy', 'Childhood Drama', 'Political cinema', 'Chinese Movies', 'Sword and Sandal', 'Gothic Film', 'Libraries and librarians', 'Romance Film', 'News', 'Juvenile Delinquency Film', "Children's", 'Adult', 'Swashbuckler films', 'Apocalyptic and post-apocalyptic fiction', 'Nature', 'Ninja movie', 'Christmas movie', 'Beach Film', 'Detective fiction', 'Master Criminal Films', 'Northern', 'Comedy film', 'Bloopers & Candid Camera', 'Crime Comedy', 'Sci-Fi Thriller', 'Sexploitation', 'Pornographic movie', 'B-movie', 'Comdedy', 'Fairy tale', 'Cyberpunk', 'Costume drama', 'Supermarionation', 'Black comedy', 'Space opera', 'Revisionist Western', 'Gay Interest', 'Drama', 'Mystery', 'Cavalry Film', 'S

In [334]:
values_to_find = ['Anime', 'Animation', 'Computer Animation', 'Clay animation', 'Animated cartoon','Stop motion']
movies = movies[movies['Movie genres'].apply(lambda x: not(any(value in x for value in values_to_find)))]

In [335]:
movies.head()

,Wikipedia Movie ID,Movie name,Movie release date,Box office revenue,Movie runtime,Movie countries,Movie genres,Movie release year
0,975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001
7,10408933,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,[United States of America],"[Musical, Comedy, Black-and-white]",1938
13,171005,Henry V,1989-11-08,10161099.0,137.0,[United Kingdom],"[Costume drama, War film, Epic, Period piece, ...",1989
17,77856,Mary Poppins,1964-08-27,102272727.0,139.0,[United States of America],"[Children's/Family, Musical, Fantasy, Comedy, ...",1964
21,612710,New Rose Hotel,1999-10-01,21521.0,92.0,[United States of America],"[Thriller, Science Fiction, Future noir, Indie...",1999


#### Ethnicity data

Let's see what the ethnicity dataset looks like.

In [336]:
ethnicities_original.head()

,Ethnicity ID,Ethnicity
0,/m/044038p,NaN
1,/m/0x67,African Americans
2,/m/064b9n,Omaha people
3,/m/041rx,Jewish people
4,/m/033tf_,Irish Americans


Here again we have NaN values for some ethnicities, we can drop them now.

In [337]:
ethnicities = ethnicities_original.copy()
ethnicities = ethnicities[ethnicities['Ethnicity'].notna()]

In [338]:
ethnicities.head()

,Ethnicity ID,Ethnicity
1,/m/0x67,African Americans
2,/m/064b9n,Omaha people
3,/m/041rx,Jewish people
4,/m/033tf_,Irish Americans
5,/m/04gfy7,Indian Americans


#### Grouping ethnicities in ethnic groups 

In [339]:
nb_ethnicities = len(np.unique(ethnicities['Ethnicity']))
print('Number of ethnicities:',nb_ethnicities)

Number of ethnicities: 431


As we can see we have a lot of different ethnicities in the dataframe. For our analysis it could be interresting to group them according to [UK's list of ethnic groups ](https://www.ethnicity-facts-figures.service.gov.uk/style-guide/ethnic-groups). Moreover to have clusters of similar size, we have decided to combine `Asian or Asian British` with `Other ethnic group` to a larger group called `Asian, Middle East and Tribes`.

In [340]:
ethnic_group = ethnic_group_original.copy()
ethnic_group.head()

,Ethnicity,Ethnic Group
0,Aceh,"Asian, Middle East and Tribes"
1,Afghans in India,"Asian, Middle East and Tribes"
2,Agrawal,"Asian, Middle East and Tribes"
3,Asian people,"Asian, Middle East and Tribes"
4,Bengali Brahmins,"Asian, Middle East and Tribes"


### Merging datafames

Now that all the datasets are cleaned, we can merge them in one big dataframe.

In [341]:
characters_extended = characters.copy().merge(ethnicities, how='inner', on='Ethnicity ID')
characters_extended.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,Ethnicity
0,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans
1,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,African Americans
2,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp,African Americans
3,156558,2001-06-27,Yvette,1970-09-11,F,/m/0x67,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,African Americans
4,156558,2001-06-27,Jody,1978-12-30,M,/m/0x67,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,African Americans


In [342]:
characters_extended = characters_extended.merge(ethnic_group, how='inner', on='Ethnicity')
characters_extended.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,Ethnicity,Ethnic Group
0,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans,"Black, Caribbean or African"
1,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,African Americans,"Black, Caribbean or African"
2,3196793,2000-02-16,NaN,1937-11-10,M,/m/0x67,62.0,/m/0lr37dy,NaN,/m/01lntp,African Americans,"Black, Caribbean or African"
3,156558,2001-06-27,Yvette,1970-09-11,F,/m/0x67,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,African Americans,"Black, Caribbean or African"
4,156558,2001-06-27,Jody,1978-12-30,M,/m/0x67,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,African Americans,"Black, Caribbean or African"


In [343]:
characters_movies = characters_extended.copy().merge(movies, how='inner', on=['Wikipedia Movie ID','Movie release date'])
characters_movies.head()

,Wikipedia Movie ID,Movie release date,Character name,Birth,Gender,Ethnicity ID,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID,Ethnicity,Ethnic Group,Movie name,Box office revenue,Movie runtime,Movie countries,Movie genres,Movie release year
0,975900,2001-08-24,Desolation Williams,1969-06-15,M,/m/0x67,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l,African Americans,"Black, Caribbean or African",Ghosts of Mars,14010832.0,98.0,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001
1,975900,2001-08-24,Commander Helena Braddock,1949-05-26,F,/m/0x67,52.0,/m/02vdcfp,/m/0bgchnd,/m/0418ft,African Americans,"Black, Caribbean or African",Ghosts of Mars,14010832.0,98.0,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001
2,975900,2001-08-24,Tres,1959-03-09,M,/m/064b9n,42.0,/m/0bgchrs,/m/0bgchrw,/m/03ydsb,Omaha people,"Asian, Middle East and Tribes",Ghosts of Mars,14010832.0,98.0,[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",2001
3,156558,2001-06-27,Yvette,1970-09-11,F,/m/0x67,30.0,/m/0jtx5t,/m/03jnxj_,/m/0blbxk,African Americans,"Black, Caribbean or African",Baby Boy,29381649.0,123.0,[United States of America],"[Crime Fiction, Drama, Coming of age]",2001
4,156558,2001-06-27,Jody,1978-12-30,M,/m/0x67,22.0,/m/0jtx5h,/m/03jnxf4,/m/01l1b90,African Americans,"Black, Caribbean or African",Baby Boy,29381649.0,123.0,[United States of America],"[Crime Fiction, Drama, Coming of age]",2001


### Showing the feasability of the project 

In this section, we are only interested in the number of different films, actors and characters we have in the processed dataframe. This is to determine whether the dataframe contains enough data to be analyzed.

In [344]:
nb_movies = len(np.unique(characters_movies['Wikipedia Movie ID']))
print("Number of movies:",nb_movies)

Number of movies: 6503


In [345]:
nb_actors = len(np.unique(characters_movies['Freebase actor ID']))
print("Number of actors:",nb_actors)

Number of actors: 4321


In [346]:
nb_characters = np.shape(characters_movies)[0]
print("Number of characters:",nb_characters)

Number of characters: 27971


In [347]:
characters_movies['Ethnic Group'].value_counts()

Ethnic Group
White                              12764
Mixed or multiple ethnic groups     5931
Asian, Middle East and Tribes       4735
Black, Caribbean or African         4541
Name: count, dtype: int64